# Dataset Join and Cleaning

Bu notebook quyidagilarni bajaradi:
1. CSV fayllarni tekshirish va validatsiya qilish
2. Ma'lumotlarni tozalash (currency, commas, categorical)
3. Barcha datasetlarni birlashtirish (join)
4. Final tozalangan datasetni saqlash


In [37]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

data_folder = Path('../converted_csv/')


In [38]:
# CSV fayllarni yuklash va validatsiya qilish
csv_files = {
    'application_metadata': 'application_metadata.csv',
    'demographics': 'demographics.csv',
    'credit_history': 'credit_history.csv',
    'loan_details': 'loan_details.csv',
    'financial_ratios': 'financial_ratios.csv',
    'geographic_data': 'geographic_data.csv'
}

datasets = {}

for name, filename in csv_files.items():
    filepath = data_folder / filename
    if not filepath.exists():
        continue
    
    try:
        df = pd.read_csv(filepath)
        datasets[name] = df
    except Exception as e:
        continue


In [39]:
# Tozalash funksiyalari
def clean_currency(value):
    """Currency va vergullarni olib tashlash: '$17,700' → 17700.0"""
    if pd.isna(value):
        return np.nan
    
    # String ga o'tkazish
    if isinstance(value, (int, float)):
        return float(value)
    
    value_str = str(value).strip()
    
    # $ belgisini olib tashlash
    value_str = value_str.replace('$', '')
    
    # Vergullarni olib tashlash
    value_str = value_str.replace(',', '')
    
    # Bo'sh joylarni olib tashlash
    value_str = value_str.strip()
    
    # Raqamga o'tkazish
    try:
        return float(value_str)
    except:
        return np.nan


def standardize_loan_type(value):
    """Qarz turini standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    if 'personal' in value_str.lower():
        return 'Personal'
    elif 'mortgage' in value_str.lower():
        return 'Mortgage'
    elif 'credit' in value_str.lower() and 'card' in value_str.lower():
        return 'CreditCard'
    elif 'auto' in value_str.lower():
        return 'Auto'
    else:
        return value_str

def standardize_education(value):
    """Ta'lim darajasini standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    # Case standartlashtirish
    if 'graduate' in value_str.lower():
        return 'Graduate'
    elif 'bachelor' in value_str.lower():
        return 'Bachelor'
    elif 'high school' in value_str.lower() or 'highschool' in value_str.lower():
        return 'High School'
    elif 'some college' in value_str.lower() or 'somecollege' in value_str.lower():
        return 'Some College'
    elif 'master' in value_str.lower():
        return 'Master'
    elif 'doctorate' in value_str.lower() or 'phd' in value_str.lower():
        return 'Doctorate'
    else:
        return value_str

def standardize_marital_status(value):
    """Oilaviy holatni standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    if 'married' in value_str.lower():
        return 'Married'
    elif 'single' in value_str.lower():
        return 'Single'
    elif 'divorced' in value_str.lower():
        return 'Divorced'
    elif 'widowed' in value_str.lower():
        return 'Widowed'
    else:
        return value_str




In [40]:
# Datasetlarni tozalash
cleaned_datasets = {}

# APPLICATION_METADATA
df_app = datasets['application_metadata'].copy()
cleaned_datasets['application_metadata'] = df_app

# DEMOGRAPHICS
df_demo = datasets['demographics'].copy()
df_demo['annual_income'] = df_demo['annual_income'].apply(clean_currency)
df_demo['education'] = df_demo['education'].apply(standardize_education)
df_demo['marital_status'] = df_demo['marital_status'].apply(standardize_marital_status)
if df_demo['employment_length'].isnull().sum() > 0:
    df_demo['employment_length'] = df_demo['employment_length'].fillna(df_demo['employment_length'].median())
cleaned_datasets['demographics'] = df_demo

# CREDIT_HISTORY
df_credit = datasets['credit_history'].copy()
if df_credit['num_delinquencies_2yrs'].isnull().sum() > 0:
    df_credit['num_delinquencies_2yrs'] = df_credit['num_delinquencies_2yrs'].fillna(0)
cleaned_datasets['credit_history'] = df_credit

# LOAN_DETAILS
df_loan = datasets['loan_details'].copy()
df_loan['loan_amount'] = df_loan['loan_amount'].apply(clean_currency)
df_loan['loan_type'] = df_loan['loan_type'].apply(standardize_loan_type)
cleaned_datasets['loan_details'] = df_loan

# FINANCIAL_RATIOS
df_fin = datasets['financial_ratios'].copy()
currency_columns = [
    'monthly_income', 'existing_monthly_debt', 'monthly_payment',
    'revolving_balance', 'credit_usage_amount', 'available_credit',
    'total_monthly_debt_payment', 'total_debt_amount', 'monthly_free_cash_flow'
]
for col in currency_columns:
    if col in df_fin.columns:
        df_fin[col] = df_fin[col].apply(clean_currency)
if 'revolving_balance' in df_fin.columns and df_fin['revolving_balance'].isnull().sum() > 0:
    df_fin['revolving_balance'] = df_fin['revolving_balance'].fillna(0)
cleaned_datasets['financial_ratios'] = df_fin

# GEOGRAPHIC_DATA
df_geo = datasets['geographic_data'].copy()
cleaned_datasets['geographic_data'] = df_geo


In [41]:
# Datasetlarni birlashtirish (JOIN)
df_final = cleaned_datasets['application_metadata'].copy()

df_final = df_final.merge(
    cleaned_datasets['demographics'],
    left_on='customer_ref',
    right_on='cust_id',
    how='left',
    suffixes=('', '_demo')
)

df_final = df_final.merge(
    cleaned_datasets['credit_history'],
    left_on='customer_ref',
    right_on='customer_number',
    how='left',
    suffixes=('', '_credit')
)

df_final = df_final.merge(
    cleaned_datasets['loan_details'],
    left_on='customer_ref',
    right_on='customer_id',
    how='left',
    suffixes=('', '_loan')
)

df_final = df_final.merge(
    cleaned_datasets['financial_ratios'],
    left_on='customer_ref',
    right_on='cust_num',
    how='left',
    suffixes=('', '_financial')
)

df_final = df_final.merge(
    cleaned_datasets['geographic_data'],
    left_on='customer_ref',
    right_on='id',
    how='left',
    suffixes=('', '_geo')
)


In [42]:
# Takrorlangan va foydasiz ustunlarni o'chirish
columns_to_drop = [
    'cust_id',
    'customer_number',
    'customer_id',
    'cust_num',
    'id',
    'recent_inquiry_count',
    'random_noise_1'
]

existing_drop_cols = [col for col in columns_to_drop if col in df_final.columns]
if existing_drop_cols:
    df_final = df_final.drop(columns=existing_drop_cols)


In [43]:
# Final validatsiya
output_file = Path('final_dataset.csv')
df_final.to_csv(output_file, index=False)


In [44]:
# Qo'shimcha tozalash funksiyalari
def clean_employment_type(value):
    if pd.isna(value):
        return value
    value_str = str(value).strip().lower()
    if value_str in ['full-time', 'full time', 'fulltime', 'ft', 'full_time']:
        return 'Full-time'
    elif value_str in ['part-time', 'part time', 'parttime', 'pt', 'part_time']:
        return 'Part-time'
    elif value_str in ['self-employed', 'self employed', 'selfemp', 'self emp', 'self-emp', 'self_employed']:
        return 'Self-employed'
    elif value_str in ['contractor', 'contract']:
        return 'Contractor'
    return str(value).strip()

def clean_education(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if value_lower in ['some college', 'high school', 'high_school']:
        return 'Low Education'
    elif value_lower in ['bachelor']:
        return 'Medium Education'
    elif value_lower in ['graduate', 'advanced']:
        return 'High Education'
    return value

def clean_marital_status(value):
    if pd.isna(value):
        return value
    return 'Married' if 'married' in str(value).lower() else 'Not Married'

def clean_loan_type(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if 'cc' in value_lower or 'credit' in value_lower and 'card' in value_lower:
        return 'CreditCard'
    elif 'mortgage' in value_lower:
        return 'Home Loan'
    return value

def clear_origination_channel(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if value_lower in ['online', 'direct mail']:
        return 'Digital'
    elif value_lower in ['branch', 'broker']:
        return 'Physical'
    return value

def convert_marital_status(value):
    return 1 if value == 'Married' else 0

def convert_origination_channel(value):
    return 1 if value == 'Digital' else 0

# Tozalash funksiyalarini qo'llash
df_final['employment_type'] = df_final['employment_type'].apply(clean_employment_type)
df_final['education'] = df_final['education'].apply(clean_education)
df_final['marital_status'] = df_final['marital_status'].apply(clean_marital_status)
df_final['loan_type'] = df_final['loan_type'].apply(clean_loan_type)
df_final['origination_channel'] = df_final['origination_channel'].apply(clear_origination_channel)
df_final['marital_status'] = df_final['marital_status'].apply(convert_marital_status)
df_final['origination_channel'] = df_final['origination_channel'].apply(convert_origination_channel)

# Final datasetni saqlash
output_file = Path('final_dataset.csv')
df_final.to_csv(output_file, index=False)

In [45]:
# Foydasiz ustunlarni o'chirish va for_model papkasiga saqlash
columns_to_drop = ['state', 'marketing_campaign']
existing_drop_cols = [col for col in columns_to_drop if col in df_final.columns]

if existing_drop_cols:
    df_final_cleaned = df_final.drop(columns=existing_drop_cols)
else:
    df_final_cleaned = df_final.copy()

# for_model papkasiga saqlash
for_model_folder = Path('../for_model')
for_model_folder.mkdir(exist_ok=True)
output_file = for_model_folder / 'dataset_for_models.csv'
df_final_cleaned.to_csv(output_file, index=False)
